In [1]:
import  numpy                   as      np
import  matplotlib.pyplot       as      plt

from    kinetic.collision_kernel import  *
from    kinetic.utils            import  *
from    kinetic.distribution     import  *
from    kinetic.parsers          import  *

In [2]:
fsm_file_list = [f'data/FSM_3D__BKW__res{int(2**i)}_Tmin5.50_Tmax10.00_dt0.10_alpha0.00.npz' for i in (3, 4, 5)]
dsm_file_list = [f'data/DSM_3D__BKW__res{int(2**i)}_Tmin5.50_Tmax10.00_dt0.10_alpha0.00.npz' for i in (3, 4)]


fsm_data_list = [np.load(file, allow_pickle=True) for file in fsm_file_list]
dsm_data_list = [np.load(file, allow_pickle=True) for file in dsm_file_list]


fsm_sol_list:   list[np.ndarray] = [x['arr_distribution'] for x in fsm_data_list]
dsm_sol_list:   list[np.ndarray] = [x['arr_distribution'] for x in dsm_data_list]
fsm_args_list:  list[SpectralMethodArguments] = [x['args'].item() for x in fsm_data_list]
dsm_args_list:  list[SpectralMethodArguments] = [x['args'].item() for x in fsm_data_list]



fsm_min_t_list: list[float] = [x.min_t for x in fsm_args_list]
fsm_max_t_list: list[float] = [x.max_t for x in fsm_args_list]
fsm_num_t_list: list[int]   = [x.num_t for x in fsm_args_list]
fsm_arr_t_list: list[np.ndarray] = [
    np.linspace(fsm_min_t_list[k], fsm_max_t_list[k], fsm_num_t_list[k])
    for k in range(len(fsm_file_list))
]
dsm_min_t_list: list[float] = [x.min_t for x in dsm_args_list]
dsm_max_t_list: list[float] = [x.max_t for x in dsm_args_list]
dsm_num_t_list: list[int]   = [x.num_t for x in dsm_args_list]
dsm_arr_t_list: list[np.ndarray] = [
    np.linspace(dsm_min_t_list[k], dsm_max_t_list[k], dsm_num_t_list[k])
    for k in range(len(dsm_file_list))
]

fsm_max_v_list:     list[float] = [args.max_v for args in fsm_args_list]
fsm_res_v_list:     list[int]   = [args.resolution for args in fsm_args_list]
fsm_grid_v_list:    list[np.ndarray] = [
    velocity_grid(3, res, v)
    for res, v in zip(fsm_res_v_list, fsm_max_v_list)
]
dsm_max_v_list:     list[float] = [args.max_v for args in dsm_args_list]
dsm_res_v_list:     list[int]   = [args.resolution for args in dsm_args_list]
dsm_grid_v_list:    list[np.ndarray] = [
    velocity_grid(3, res, v)
    for res, v in zip(dsm_res_v_list, dsm_max_v_list)
]

In [3]:
fsm_bkw_list:  list[np.ndarray] = [
    bkw(arr_t, v, args.bkw_coeff_ext, args.vhs_coeff)
    for arr_t, v, args in zip(fsm_arr_t_list, fsm_grid_v_list, fsm_args_list)
]
dsm_bkw_list:  list[np.ndarray] = [
    bkw(arr_t, v, args.bkw_coeff_ext, args.vhs_coeff)
    for arr_t, v, args in zip(dsm_arr_t_list, dsm_grid_v_list, dsm_args_list)
]

----------[ BKW solution ]----------
* coeff_ext: 1
* coeff_int: 0.16666666666666666
------------------------------------
----------[ BKW solution ]----------
* coeff_ext: 1
* coeff_int: 0.16666666666666666
------------------------------------
----------[ BKW solution ]----------
* coeff_ext: 1
* coeff_int: 0.16666666666666666
------------------------------------
----------[ BKW solution ]----------
* coeff_ext: 1
* coeff_int: 0.16666666666666666
------------------------------------
----------[ BKW solution ]----------
* coeff_ext: 1
* coeff_int: 0.16666666666666666
------------------------------------


In [4]:
fsm_bkw_list[0].shape

(46, 8, 8, 8)

In [5]:
METRIC_ORDER = float('inf')
fsm_error_list = [metric(fsm_sol_list[k][0], fsm_bkw_list[k], METRIC_ORDER) for k in range(len(fsm_file_list))]
dsm_error_list = [metric(dsm_sol_list[k][0], dsm_bkw_list[k], METRIC_ORDER) for k in range(len(dsm_file_list))]

fsm_fmt_list = ['r--', 'g--', 'b--']
dsm_fmt_list = ['r-', 'g-']

plt.figure(figsize=(10,6))
plt.title(f"Relative error of order {METRIC_ORDER}")
for k in range(len(fsm_file_list)):
    plt.plot(fsm_arr_t_list[k], fsm_error_list[k], fsm_fmt_list[k], label=f"Fast, $N={2**(k+3)}$")
for k in range(len(dsm_file_list)):
    plt.plot(dsm_arr_t_list[k], dsm_error_list[k], dsm_fmt_list[k], label=f"Direct, $N={2**(k+3)}$")
plt.yscale('log')
plt.ylim((1e-8, 1e+1))
plt.xlim((5.6, 8.5))
plt.legend()
plt.grid(True)
plt.show()

TypeError: linalg_vector_norm(): argument 'dim' (position 3) must be tuple of ints, not float

End of file

In [9]:
axes = tuple(range(1, fsm_sol_list[0].ndim))
print(axes[0].__class__)

<class 'int'>


In [ ]:
X = dsm_bkw_list[1] - fsm_bkw_list[1]

In [ ]:
plt.imshow(dsm_bkw_list[1][10, 8])

In [ ]:
print(dsm_args_list[1])